In [23]:
# download requirement
!pip install kaggle
!pip install panda
!pip install matplotlib

  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=6a703d498c2c0a5a68df41d0938e2c151df458e2c5d1e620a9e44adafa41e6dd
  Stored in directory: /home/onyxia/.cache/pip/wheels/df/5c/39/36f8dae25a1e88d6ec4411dec4a143781e64fdff6897758eec
Successfully built panda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 20.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.5 MB/s eta 0:00:00


In [22]:
# download dataset zip
!kaggle datasets download jutrera/stanford-car-dataset-by-classes-folder
# create folder data
!mkdir data
# unzip data into data file
!mv stanford-car-dataset-by-classes-folder.zip data
%cd data
!unzip stanford-car-dataset-by-classes-folder.zip
%cd ../

mkdir: cannot create directory ‘data’: File exists
mv: cannot stat 'stanford-car-dataset-by-classes-folder.zip': No such file or directory
/home/onyxia/work/similarite-hackathon/data
Archive:  stanford-car-dataset-by-classes-folder.zip
replace anno_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
/home/onyxia/work/similarite-hackathon
